<img src=.\Img\sci.png>

### Финансовый университет

##  Инструментальная поддержка анализа финансово-экономических данных

## Тема 6. Работа с базами данных в Пайтон: объектно-реляционное сопоставление

Лекция<br>
15 мая 2021 года<br>
Поток: ПМ18-1, ПМ18-2, ПМ18-3, ПМ18-4

Преподаватель: Смирнов Михаил Викторович, доцент Департамента анализа данных и машинного обучения Финансового университета при Правительстве Российской Федерации. mvsmirnov@fa.ru

Москва - 2021

При подготовке материалов учебных занятий использовались источники
- Essential SQLAlchemy: Mapping Python to Databases 2nd Edition. Jason Myers, Rick Copeland. O'Reilly Media, Inc. 2015.
- Астахова И.Ф., Мельников В.М., Толстобров А.П., Фертиков В.В. СУБД: язык SQL в примерах и задачах.—М.:ФИЗМАТЛИТ, 2009. — 168 с. — ISBN 978-5-9221-0816-4.

В ряде учебных примеров использованы данные <a href="http://insideairbnb.com/get-the-data.html">Inside Airbnb</a>
    


<a id=Ref></a>
# Оглавление

[Введение](#Intro)<br>
[6.1. Определение таблиц через классы ORM](#T_3_1)<br>
[6.2. Сеанс](#T_3_2)<br>
[6.3. Вставка данных](#T_3_3)<br>
[6.4. Запросы](#T_3_4)<br>
[6.5. Отображение](#T_3_5)<br>
[6.6. Сеанс](#T_3_6)<br>


<a id=Intro></a>
# Введение
[<= ](#Ref)||[ К оглавлению ](#Ref)||[ =>](#T_3_1)

ORM - Object-relational mapping - Объектно-реляционное сопоставление (отображение)

SQLAlchemy ORM обеспечивает эффективный способ привязки схемы и операций базы данных к объектам данных.

В SQLAlchemy Core мы создавали контейнер метаданных, а затем объявляли объект *Table*, связанный с этими метаданными. В SQLAlchemy ORM мы будем определять класс, который наследуется от специального базового класса *declarative_base*. Этот базовый класс объединяет контейнер метаданных и средство сопоставления, которое сопоставляет наш класс с таблицей базы данных. Он также сопоставляет экземпляры класса с записями в этой таблице.

<a id=T_3_1></a>
[<= ](#Intro)||[ К оглавлению ](#Ref)||[ =>](#T_3_2)

# 6.1. Определение таблиц через классы ORM

ORM классы должны:
- Происходить от класса *declarative_base*.
- Содержать `__tablename__`, которое является именем таблицы базы данных.
- Содержать один или несколько атрибутов, которые являются объектами *Column*.
- Содержать атрибуты, составляющие первичный ключ.

Изучим требование, связанное с атрибутами. Определение столбцов в классе ORM похоже на определение столбцов в объекте *Table*, которое мы изучили в теме SQLAlchemy Core. Однако есть важное отличие. При определении столбцов в классе ORM в качестве имени столбца будет установлено имя атрибута класса, которому он назначен. Все остальное, что связано с типами данных и столбцами, применимо и здесь.

<img src="./Img/Listings_ORM_Schema.png">

<br><br>
Определим таблицу *listings* как класс ORM

In [1]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import (Table, Column, 
                        Integer, Numeric, String, Boolean,
                        ForeignKey, ForeignKeyConstraint, CheckConstraint)

from datetime import datetime
from sqlalchemy import DateTime

Base = declarative_base()

In [2]:
class Listing(Base):
    __tablename__ = 'listings'

    listing_id = Column(Integer(), primary_key = True)
    listing_name = Column(String(50), index = True, nullable = False)
    listing_url = Column(String(50))
    host_id = Column(Integer())
    neighbourhood_id = Column(Integer())
    amenities = Column(String(250))
    property_type_id = Column(Integer())
    room_type_id = Column(Integer())
    bedrooms = Column(Integer())
    beds = Column(Integer())
    price = Column('price',Numeric(7,2))
    
    __table_args__ = (
        ForeignKeyConstraint(['neighbourhood_id'],['neighbourhoods.neigh_id']),
        ForeignKeyConstraint(['property_type_id'], ['property_types.property_type_id']),
        ForeignKeyConstraint(['room_type_id'], ['room_types.room_type_id']),
        CheckConstraint('price >= 0.00', name='listing_price_positive')
    )
    

В этом примере *Base* - экземпляр класса *declarative_base()*. Затем создается дочерний класс *Listings*. Определяется имя таблицы 'listings'. Определяются атрибуты, устанавливается первичный ключ. Обратимся к свойству `__table__` класса.

In [3]:
Listing.__table__

Table('listings', MetaData(bind=None), Column('listing_id', Integer(), table=<listings>, primary_key=True, nullable=False), Column('listing_name', String(length=50), table=<listings>, nullable=False), Column('listing_url', String(length=50), table=<listings>), Column('host_id', Integer(), table=<listings>), Column('neighbourhood_id', Integer(), ForeignKey('neighbourhoods.neigh_id'), table=<listings>), Column('amenities', String(length=250), table=<listings>), Column('property_type_id', Integer(), ForeignKey('property_types.property_type_id'), table=<listings>), Column('room_type_id', Integer(), ForeignKey('room_types.room_type_id'), table=<listings>), Column('bedrooms', Integer(), table=<listings>), Column('beds', Integer(), table=<listings>), Column('price', Numeric(precision=7, scale=2), table=<listings>), schema=None)

Создадим класс для клиентов

In [4]:
from datetime import datetime
from sqlalchemy import DateTime

class User(Base):
    __tablename__ = 'users'
    
    user_id = Column(Integer(), primary_key = True)
    username = Column(String(15), nullable = False, unique = True)
    email_address = Column(String(255), nullable = False)
    phone = Column(String(20), nullable = False)
    password = Column(String(25), nullable = False)
    created_on = Column(DateTime(), default = datetime.now)
    updated_on = Column(DateTime(), default = datetime.now, onupdate=datetime.now)

Здесь мы определили несколько атрибутов, которые не могут оставаться пустыми. Требуется уникальное значение *username*. Для атрибута *updated_on* мы установили текущее время по умолчанию, если время не указано. Использование *onupdate* приведет к установке текущего времени при обновлении любого атрибута записи.

## 6.1.1. Ключи, ограничения, индексы

Ранее, в разделе *Core*, мы изучили, что ключи и ограничения могут задаваться как в составе элемента *Column()* конструктора *Table*, так и в явном виде. Например, в *line_items* атрибут *order_id* является внешним ключом, тода

`ForeignKeyConstraint(['order_id'], ['order.order_id'])`. 

В ORM также существует для этого два способа, но так как конструктор *Table* здесь не используется, то применяются свойства класса. 

```
user_id = Column(Integer(), ForeignKey('users.user_id'))
```

Для задания ограничения в явном виде в классе используется `__table_args__`

```
class SomeDataClass(Base):
    __tablename__ = 'somedatatable'
    __table_args__ = (ForeignKeyConstraint(['id'], ['other_table.id']),
                      CheckConstraint(price >= 0.00', name='unit_cost_positive'))
```
В данном примере значением `__table_args__` является кортеж.

### Задание 6.1.1.1.
Создать классы *Order, Line_item, Host, Neighbourhood, Room_type, Property_type*. Создать базу данных *Listings.db*

In [5]:
# Ваш код здесь


In [6]:
class Order(Base):

    __tablename__ = 'orders'
    order_id = Column(Integer(), primary_key = True)
    user_id = Column(Integer())
    
    __table_args__ = (ForeignKeyConstraint(['user_id'], ['users.user_id']),)

In [7]:
Order.__table__

Table('orders', MetaData(bind=None), Column('order_id', Integer(), table=<orders>, primary_key=True, nullable=False), Column('user_id', Integer(), ForeignKey('users.user_id'), table=<orders>), schema=None)

In [8]:
class Line_item(Base):
    
    __tablename__ = 'line_items'
    item_id = Column(Integer(), primary_key = True)
    order_id = Column(Integer(), ForeignKey('orders.order_id'))
    listing_id = Column(Integer(), ForeignKey('listings.listing_id'))
    item_start_date = Column(DateTime(), nullable = False, default = datetime.now)
    item_end_date = Column('item_end_date', DateTime(), nullable = False)

In [9]:
Line_item.__table__

Table('line_items', MetaData(bind=None), Column('item_id', Integer(), table=<line_items>, primary_key=True, nullable=False), Column('order_id', Integer(), ForeignKey('orders.order_id'), table=<line_items>), Column('listing_id', Integer(), ForeignKey('listings.listing_id'), table=<line_items>), Column('item_start_date', DateTime(), table=<line_items>, nullable=False, default=ColumnDefault(<function datetime.now at 0x000001E4A72E30D8>)), Column('item_end_date', DateTime(), table=<line_items>, nullable=False), schema=None)

In [10]:
class Host(Base):
    
    __tablename__ = 'hosts'
    host_id = Column(Integer(), primary_key = True)
    host_name = Column(String(50), nullable = False)

In [11]:
class Neighbourhood(Base):
    
    __tablename__ = 'neighbourhoods'
    neigh_id = Column(Integer(), primary_key = True)
    neigh_name = Column(String(50), nullable = False, unique = True)

In [12]:
class Room_type(Base):
    
    __tablename__ = 'room_types'
    room_type_id = Column(Integer(), primary_key = True)
    room_type_name = Column(String(50), nullable = False)

In [13]:
class Property_type(Base):
    
    __tablename__ = 'property_types'
    property_type_id = Column(Integer(), primary_key = True)
    property_type_name = Column(String(50), nullable = False)

## 6.1.2. Сохранение схемы

In [14]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:') # a) В памяти
#engine = create_engine('sqlite:///Listings.db') # b) На диске

Base.metadata.create_all(engine)

## 6.1.3. Связи
В *ORM* имеются некоторые различия при связывании таблиц по сравнению с *Core*. *ORM* также использует *ForeignKey* для ограничения и связывания объектов. Однако *ORM* использует директиву *relationship* чтобы предоставить свойство доступа к связанному объекту. Это добавляет некоторые накладные расходы при использовании *ORM*; однако плюсы перевешивают недостатки. В примере показано, как определить связи с помощью методов *relationship* и *backref*.
```
from sqlalchemy.orm import relationship, backref

class Orders(Base):
    __tablename__='orders'
    order_id=Column(Integer(), primary_key=True)
    user_id=Column(Integer(), ForeignKey('users.user_id'))

    User=relationship('Users', backref=backref('orders', order_by=order_id))
```
Таким образом, в классе *Orders*, устанавливается отношение «один ко многим» с классом *Users*. Мы можем связать пользователя с его заказом, обратившись к свойству *user*. Это отношение также устанавливает свойство *orders* в классе *Users* через аргумент ключевого слова *backref*, которое упорядочивается по *order_id*. Директиве *relationship* требуется целевой класс для отношения, и она может дополнительно включать обратное отношение для целевого класса. *SQLAlchemy* знает, как сопоставить заданный нами *ForeignKey* с классом, который мы определили в отношении. В этом примере команда `ForeignKey(users.user_id)` сопоставляется с классом *User* через атрибут `__tablename__` пользователей и формирует связь. В строке 
```
User=relationship('Users', backref=backref('orders', order_by=order_id))
```
устанавливается связь *один ко многим*.

Также возможно установить взаимно-однозначное отношение *один к одному*. В следующем примере класс *Line_items* имеет взаимно-однозначное отношение с классом *Listings*. Аргумент ключевого слова `uselist = False` определяет его как взаимно однозначное отношение. Здесь используется более простая обратная ссылка, поскольку нам не нужно контролировать порядок.
```
class Line_items(Base):
    
    __tablename__='line_items'
    item_id=Column(Integer(), primary_key=True)
    order_id=Column(Integer(), ForeignKey('orders.order_id'))
    listing_id=Column(Integer(), ForeignKey('listings.listing_id'))
    item_start_date=Column(DateTime(), nullable=False, default=datetime.now)
    item_end_date=Column('item_end_date', DateTime(), nullable=False)
    
    Order=relationship("Orders", backref=backref('line_items', order_by=line_item_id))
    Listing=relationship("Listings", uselist=False))
```

<a id=T_3_2></a>
[<= ](#T_3_1)||[ К оглавлению ](#Ref)||[ =>](#T_3_3)
# 6.2. Сеанс

Сеанс - это способ взаимодействия ORM SQLAlchemy с базой данных. Он "обертывает" соединение с базой данных через механизм и предоставляет карту идентификации для объектов, которые вы загружаете через сеанс или связываете с сеансом. Карта идентификации - это структура данных, подобная кешу, которая содержит уникальный список объектов, определяемый таблицей объекта и первичным ключом. Сеанс также "обертывает" транзакцию, и эта транзакция будет открыта до тех пор, пока сеанс не будет зафиксирован или не пройзойдет откат, что очень похоже на процесс, описанный в теме *Core*.

Для нового сеанса SQLAlchemy предоставляет класс *sessionmaker*, чтобы гарантировать, что сеансы могут быть созданы с одинаковыми параметрами во всем приложении. SQLAlchemy делает это путем создания класса сеанса (Session), который настроен в соответствии с аргументами, переданными в класс *sessionmaker*, который следует использовать только один раз в глобальной области действия приложения и рассматривать как параметр конфигурации. Создадим новый сеанс, связанный с базой данных SQLite в памяти:

In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker # 1 

#engine = create_engine('sqlite:///:memory:') # 2a
engine = create_engine('sqlite:///Listings.db') # 2b

Session = sessionmaker(bind=engine) # 3

session = Session() # 4

1. Импорт модуля создания сеанса *sessionmaker*.
2. База данных SQLite a) в памяти, b) на диске.
3. Определение класса сеанса с привязкой к механизму.
4. Создание сеанса.

Теперь у нас есть сеанс, который мы можем использовать для взаимодействия с базой данных. Определим классы таблиц базы данных. Дополнительно добавим методы `__repr__`, чтобы упростить просмотр и воссоздание экземпляров объектов.

In [2]:
from datetime import datetime

from sqlalchemy import (Table, Column, 
                        Integer, Numeric, String, Boolean, DateTime,
                        ForeignKey, ForeignKeyConstraint, CheckConstraint)

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref

Base = declarative_base()


class User(Base):
    __tablename__ = 'users'
    
    user_id = Column(Integer(), primary_key = True)
    username = Column(String(15), nullable = False, unique = True)
    email_address = Column(String(255), nullable = False)
    phone = Column(String(20), nullable = False)
    password = Column(String(25), nullable = False)
    created_on = Column(DateTime(), default = datetime.now)
    updated_on = Column(DateTime(), default = datetime.now, onupdate = datetime.now)

    def __repr__(self):
        return "User(username='{self.username}', " \
                     "email_address='{self.email_address}', " \
                     "phone='{self.phone}', " \
                     "password='{self.password}')".format(self=self)

    
class Order(Base):

    __tablename__ = 'orders'
    order_id = Column(Integer(), primary_key = True)
    user_id = Column(Integer())
    created_on = Column(DateTime(), default = datetime.now)
    
    __table_args__ = (ForeignKeyConstraint(['user_id'], ['users.user_id']),)
    
    User=relationship("User", backref=backref('orders', order_by=order_id))
    
    def __repr__(self):
        return "Order(user_id='{self.user_id}', " \
                    "user_id='{self.user_id}', " \
                    "created_on='{self.created_on}')".format(self=self)
    

class Line_item(Base):
    
    __tablename__ = 'line_items'
    item_id = Column(Integer(), primary_key = True)
    order_id = Column(Integer(), ForeignKey('orders.order_id'))
    listing_id = Column(Integer(), ForeignKey('listings.listing_id'))
    item_start_date = Column(DateTime(), nullable = False, default = datetime.now)
    item_end_date = Column('item_end_date', DateTime(), nullable = False)
    
    Order=relationship("Order", backref=backref('line_items', order_by=item_id))
    Listing=relationship("Listing", uselist=False)
    
    def __repr__(self):
        return "Line_item(order_id='{self.order_id}', " \
                        "listing_id='{self.listing_id}', " \
                        "item_start_date='{self.item_start_date}', " \
                        "item_end_date='{self.item_end_date}')".format(self=self)


class Host(Base):
    
    __tablename__ = 'hosts'
    host_id = Column(Integer(), primary_key = True)
    host_name = Column(String(50), nullable = False)
    def __repr__(self):
        return "Host(host_id='{self.host_name}')".format(self=self)
    
    
class Neighbourhood(Base):
    
    __tablename__ = 'neighbourhoods'
    neigh_id = Column(Integer(), primary_key = True)
    neigh_name = Column(String(50), nullable = False, unique = True)
    def __repr__(self):
        return "Neighbourhood(neigh_name='{self.neigh_name}')".format(self=self)
    
    
class Room_type(Base):
    
    __tablename__ = 'room_types'
    room_type_id = Column(Integer(), primary_key = True)
    room_type_name = Column(String(50), nullable = False)
    def __repr__(self):
        return "Room_type(room_type_name='{self.room_type_name}')".format(self=self)

### Задание 6.2.1.

Создайте класс *Property_type* для справочника типов собственности *property_types*

In [3]:
# Ваш код здесь


In [4]:
class Property_type(Base):
    
    __tablename__ = 'property_types'
    property_type_id = Column(Integer(), primary_key = True)
    property_type_name = Column(String(50), nullable = False)
    def __repr__(self):
        return "Property_type(property_type_name='{self.property_type_name}')".format(self=self)

### Задание 6.2.2.
Создайте класс *Listing* с отношениями к классам *Host, Neighbourhood, Room_type, Property_type, Line_item* и методами `__repr__`

In [5]:
# Ваш код здесь


In [6]:
class Listing(Base):
    __tablename__ = 'listings'

    listing_id = Column(Integer(), primary_key = True)
    listing_name = Column(String(50), index = True, nullable = False)
    listing_url = Column(String(50))
    host_id = Column(Integer())
    neighbourhood_id = Column(Integer())
    amenities = Column(String(250))
    property_type_id = Column(Integer())
    room_type_id = Column(Integer())
    bedrooms = Column(Integer())
    beds = Column(Integer())
    price = Column('price',Numeric(7,2))
    
    __table_args__ = (
        ForeignKeyConstraint(['neighbourhood_id'],['neighbourhoods.neigh_id']),
        ForeignKeyConstraint(['property_type_id'], ['property_types.property_type_id']),
        ForeignKeyConstraint(['room_type_id'], ['room_types.room_type_id']),
        ForeignKeyConstraint(['host_id'], ['hosts.host_id']),
        CheckConstraint('price >= 0.00', name='listing_price_positive')
    )
    
    Host=relationship('Host', backref=backref('listings', order_by=listing_id))
    Neighbourhood=relationship('Neighbourhood', backref=backref('listings', order_by=listing_id))
    Property_type=relationship('Property_type', backref=backref('listings', order_by=listing_id))
    Room_type=relationship('Room_type', backref=backref('listings', order_by=listing_id))

    
    def __repr__(self):
        return "Listing(listing_name='{self.listing_name}', " \
                       "listing_url='{self.listing_url}', " \
                       "amenities='{self.amenities}', " \
                       "bedrooms='{self.bedrooms}', " \
                       "price='{self.price}')".format(self=self)

In [7]:
Base.metadata.create_all(engine)

<a id=T_3_3></a>
[<= ](#T_3_2)||[ К оглавлению ](#Ref)||[ =>](#T_3_4)

# 6.3. Вставка данных

In [8]:
import pandas as pd

In [9]:
df = pd.read_csv('./Data/ListingsAm.csv', sep=";")
df.head(1)

,id,listing_url,name,host_id,host_name,host_is_superhost,neighbourhood_cleansed,property_type,room_type,bathrooms_text,...,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,20168,https://www.airbnb.com/rooms/20168,Studio with private bathroom in the centre 1,59484,Alexander,f,Centrum-Oost,Private room in townhouse,Private room,1 private bath,...,2010-03-02,2020-04-09,89,10.0,10.0,10.0,10.0,10.0,9.0,2.58


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3535 entries, 0 to 3534
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           3535 non-null   int64  
 1   listing_url                  3535 non-null   object 
 2   name                         3532 non-null   object 
 3   host_id                      3535 non-null   int64  
 4   host_name                    3535 non-null   object 
 5   host_is_superhost            3535 non-null   object 
 6   neighbourhood_cleansed       3535 non-null   object 
 7   property_type                3535 non-null   object 
 8   room_type                    3535 non-null   object 
 9   bathrooms_text               3534 non-null   object 
 10  bedrooms                     3366 non-null   float64
 11  beds                         3532 non-null   float64
 12  amenities                    3535 non-null   object 
 13  price             

В наборе имеется несколько строк с пустыми названиями объектов. Включение таких объектов в базу данных запрещено, так как установлено ограничение - запрет на пустое название объекта. Удалим такие строки. Сначала распечатаем их.

In [11]:
df[df['name'].isna()]

,id,listing_url,name,host_id,host_name,host_is_superhost,neighbourhood_cleansed,property_type,room_type,bathrooms_text,...,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
833,6893255,https://www.airbnb.com/rooms/6893255,NaN,4223835,Tosca,f,De Baarsjes - Oud-West,Entire condominium,Entire home/apt,1 bath,...,2015-06-21,2019-12-30,94,10.0,9.0,9.0,9.0,9.0,9.0,0.27
905,7452777,https://www.airbnb.com/rooms/7452777,NaN,13561102,Eva,f,De Baarsjes - Oud-West,Entire apartment,Entire home/apt,1 bath,...,2015-08-03,2016-09-26,97,10.0,10.0,10.0,10.0,9.0,9.0,0.11
1320,12200500,https://www.airbnb.com/rooms/12200500,NaN,65597646,Yasmine,f,De Pijp - Rivierenbuurt,Entire apartment,Entire home/apt,1 bath,...,2016-05-29,2017-06-18,100,10.0,10.0,10.0,9.0,10.0,9.0,0.11


Теперь удалим эти записи

In [12]:
to_drop_idx = list(df[df['name'].isna()].index)
df.drop(to_drop_idx, axis=0, inplace=True)

Также мы видим, что в части записей информация о bedrooms и beds отсутствует. Установим нулевые значения.

In [13]:
df['beds'].fillna(0, inplace=True)
df['bedrooms'].fillna(0, inplace=True)

Подготовим справочник районов

In [14]:
neigh_df = df["neighbourhood_cleansed"].value_counts().sort_index().reset_index()
neigh_df.index = range(1, len(neigh_df)+1)
neigh_dict = neigh_df["index"].to_dict()
neigh_dict

{1: 'Bijlmer-Centrum',
 2: 'Bos en Lommer',
 3: 'Buitenveldert - Zuidas',
 4: 'Centrum-Oost',
 5: 'Centrum-West',
 6: 'De Aker - Nieuw Sloten',
 7: 'De Baarsjes - Oud-West',
 8: 'De Pijp - Rivierenbuurt',
 9: 'Geuzenveld - Slotermeer',
 10: 'IJburg - Zeeburgereiland',
 11: 'Noord-Oost',
 12: 'Noord-West',
 13: 'Oostelijk Havengebied - Indische Buurt',
 14: 'Osdorp',
 15: 'Oud-Noord',
 16: 'Oud-Oost',
 17: 'Slotervaart',
 18: 'Watergraafsmeer',
 19: 'Westerpark',
 20: 'Zuid'}

In [15]:
for key, value in neigh_dict.items():
    w = Neighbourhood(neigh_id = key, neigh_name = value)
    session.add(w)
session.commit()

print(w.neigh_id, w.neigh_name)

20 Zuid


### Задание 6.3.1.
Наполнить данными справочники владельцев недвижимости, типов комнат и типов собственности. При создании справочника владельцев в качестве значения первичного ключа указать значение индекса владельца из *ListingsAm.csv*

Чтобы добавить запись, содержащую значения нескольких столбцов, например первичного ключа и названия, можно в одном выражении указать несколько параметров класса таблицы:

```
w=ClassName(Column_1=value_1, Column_2=value_2, ... Column_n=value_n)
sesseion.add(w)
```

In [16]:
# Ваш код здесь


In [17]:
H=df[['host_id','host_name']].groupby(["host_id", "host_name"])['host_id'].count()
H.head()

host_id  host_name      
47517    Geert Alexander    1
58458    Dre                1
59484    Alexander          1
61977    Marjolein          1
81046    Fabienne           1
Name: host_id, dtype: int64

In [18]:
for item in H.index:
    w = Host(host_id=item[0], host_name=item[1])
    session.add(w)
session.commit()
w.host_id, w.host_name

(331113200, 'Jennifer')

Проверка, сколько записей в таблице *hosts*.

In [19]:
from sqlalchemy.sql import select, func
connection=engine.connect()
s=select([func.count(Host.__table__)])
print(str(s))
rp = connection.execute(s)
rp.scalar()

SELECT count() AS count_1 
FROM hosts


3110

Наполнение справочника типов комнат

In [20]:
room_type_df = df["room_type"].value_counts().sort_index().reset_index()
room_type_df.index = range(1, len(room_type_df)+1)
room_type_dict=room_type_df["index"].to_dict()

for value in room_type_dict.values():
    w = Room_type(room_type_name = value)
    session.add(w)

session.commit()
w.room_type_id, w.room_type_name

(4, 'Shared room')

Наполнение справочников типов собственности

In [21]:
property_type_df = df["property_type"].value_counts().sort_index().reset_index()
property_type_df.index = range(1, len(property_type_df)+1)
property_type_dict = property_type_df["index"].to_dict()

for value in property_type_dict.values():
    w = Property_type(property_type_name = value)
    session.add(w)

session.commit()
w.property_type_id, w.property_type_name

(42, 'Tiny house')

### Задание 6.3.2.

Наполнить данными таблицу *listings*

In [22]:
# Ваш код здесь

In [23]:
df.head(1)

,id,listing_url,name,host_id,host_name,host_is_superhost,neighbourhood_cleansed,property_type,room_type,bathrooms_text,...,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,20168,https://www.airbnb.com/rooms/20168,Studio with private bathroom in the centre 1,59484,Alexander,f,Centrum-Oost,Private room in townhouse,Private room,1 private bath,...,2010-03-02,2020-04-09,89,10.0,10.0,10.0,10.0,10.0,9.0,2.58


В процессе работы выдается предупреждение о том, что тип данных "десятичное число" не поддерживается напрямую. Чтобы отключить такие сообщения, выполним команду
```
import warnings; warnings.simplefilter('ignore')
```

In [24]:
import warnings; warnings.simplefilter('ignore')

neigh_dict_reversed = {}
for key, value in neigh_dict.items():
    neigh_dict_reversed[value] = key

property_type_dict_reversed = {}
for key, value in property_type_dict.items():
    property_type_dict_reversed[value] = key

room_type_dict_reversed = {}
for key, value in room_type_dict.items():
    room_type_dict_reversed[value] = key
    
for row in df.index[::]:
    w = Listing(
        listing_id = int(df.loc[row, 'id']),
        listing_name = df.loc[row, 'name'],
        listing_url = df.loc[row, 'listing_url'],
        host_id = int(df.loc[row, 'host_id']),
        neighbourhood_id = int(neigh_dict_reversed[df.loc[row, 'neighbourhood_cleansed']]),
        amenities = df.loc[row, 'amenities'],
        property_type_id = int(property_type_dict_reversed[df.loc[row, 'property_type']]),
        room_type_id = int(room_type_dict_reversed[df.loc[row, 'room_type']]),
        bedrooms = int(df.loc[row, 'bedrooms']),
        beds = int(df.loc[row, 'beds']),
        price = int(df.loc[row, 'price'])
    )
    
    session.add(w)

session.commit()
print(w.listing_id, w.listing_url, w.listing_name, 
      w.host_id, w.neighbourhood_id, w.property_type_id, w.room_type_id)

45854789 https://www.airbnb.com/rooms/45854789 The Weber Collection - Luxurieus Design Studio - 8 178187873 4 11 1


## Наполнение таблиц *users, orders, line_items*

Таблица *users*

In [25]:
User.__table__

Table('users', MetaData(bind=None), Column('user_id', Integer(), table=<users>, primary_key=True, nullable=False), Column('username', String(length=15), table=<users>, nullable=False), Column('email_address', String(length=255), table=<users>, nullable=False), Column('phone', String(length=20), table=<users>, nullable=False), Column('password', String(length=25), table=<users>, nullable=False), Column('created_on', DateTime(), table=<users>, default=ColumnDefault(<function datetime.now at 0x0000024A45355E58>)), Column('updated_on', DateTime(), table=<users>, onupdate=ColumnDefault(<function datetime.now at 0x0000024A45355288>), default=ColumnDefault(<function datetime.now at 0x0000024A45355048>)), schema=None)

In [26]:
user_list=[(1,'Nicolas','nicolas@rambler.ru','+7-929-616-88-77','@#$%890'),
           (2,'Lida','lidaok@gmail.com','+7-929-616-88-77','yyT$%333'),
           (3,'Vera','lveramuns@gmail.com','+7-353-214-12-90','yyT$%333'),
           (4,'Ivan','ivaturgenev@yandex.ru','+7-047-121-89-95','tT6^7&#20Oy'),
           (5,'Svetlana','svetaivanova@microsoft.com','+7-812-555-48-71','SD%@OUsdc7')
          ]

for item in user_list:
    w = User(
        user_id=item[0],
        username=item[1],
        email_address=item[2],
        phone=item[3],
        password=item[4],
        created_on=datetime.now()
    )
    session.add(w)
session.commit()

print(w.user_id, w.username, w.email_address, 
      w.phone, w.password, w.created_on, w.updated_on)

5 Svetlana svetaivanova@microsoft.com +7-812-555-48-71 SD%@OUsdc7 2021-05-12 20:30:52.611917 2021-05-12 20:30:52.618951


Таблица *orders*

In [27]:
Order.__table__

Table('orders', MetaData(bind=None), Column('order_id', Integer(), table=<orders>, primary_key=True, nullable=False), Column('user_id', Integer(), ForeignKey('users.user_id'), table=<orders>), Column('created_on', DateTime(), table=<orders>, default=ColumnDefault(<function datetime.now at 0x0000024A45331678>)), schema=None)

In [28]:
order_list = [(1,1,'2020-05-01'),(2,1,'2020-05-02'),
              (3,2,'2020-05-03'),(4,3,'2020-05-04'),
              (5,3,'2020-05-05'),(6,4,'2020-05-06'),
              (7,4,'2020-05-07'),(8,4,'2020-05-08'),
              (9,5,'2020-05-09'),(10,5,'2020-05-10'),
              (11,5,'2021-05-09'),(12,5,'2021-05-09')]

for item in order_list:
    w = Order(
        order_id=item[0],
        user_id=item[1],
        created_on=datetime.strptime(item[2], '%Y-%m-%d')
    )
    session.add(w)
session.commit()

print(w.order_id, w.user_id, w.created_on)

12 5 2021-05-09 00:00:00


Таблица *line_items*

In [29]:
Line_item.__table__

Table('line_items', MetaData(bind=None), Column('item_id', Integer(), table=<line_items>, primary_key=True, nullable=False), Column('order_id', Integer(), ForeignKey('orders.order_id'), table=<line_items>), Column('listing_id', Integer(), ForeignKey('listings.listing_id'), table=<line_items>), Column('item_start_date', DateTime(), table=<line_items>, nullable=False, default=ColumnDefault(<function datetime.now at 0x0000024A45331438>)), Column('item_end_date', DateTime(), table=<line_items>, nullable=False), schema=None)

In [30]:
line_list=[(1,1,20168,'2020-05-07','2020-05-17'),
           (2,1,27886,'2020-05-17','2020-05-27'),
           (3,2,28871,'2020-08-01','2020-08-10'),
           (4,3,29051,'2020-02-06','2020-03-17'),
           (5,4,2550571,'2020-04-07','2020-06-01'),
           (6,4,18225467,'2020-05-07','2020-06-01'),
           (7,5,3908795,'2020-07-07','2020-08-19'),
           (8,6,16550704,'2020-08-03','2020-08-11'),
           (9,7,3908795,'2020-09-02','2020-09-12'),
          ]

for jtem in line_list:
    w = Line_item(
        item_id=jtem[0],
        order_id=jtem[1],
        listing_id=jtem[2],
        item_start_date=datetime.strptime(jtem[3],'%Y-%m-%d'),
        item_end_date=datetime.strptime(jtem[4],'%Y-%m-%d')
    )
    session.add(w)
session.commit()

print(w.item_id, w.order_id, w.listing_id, w.item_start_date, w.item_end_date)

9 7 3908795 2020-09-02 00:00:00 2020-09-12 00:00:00


<a id=T_3_4></a>
[<= ](#T_3_3)||[ К оглавлению ](#Ref)||[ =>](#T_3_5)
# 6.4. Запросы

### Методы *first(), all(), one().* Ограничение *limit()*.

In [31]:
session.query(Room_type).all()

[Room_type(room_type_name='Entire home/apt'),
 Room_type(room_type_name='Hotel room'),
 Room_type(room_type_name='Private room'),
 Room_type(room_type_name='Shared room')]

In [32]:
session.query(Property_type).limit(3).all()

[Property_type(property_type_name='Boat'),
 Property_type(property_type_name='Entire apartment'),
 Property_type(property_type_name='Entire bed and breakfast')]

In [33]:
session.query(Neighbourhood.neigh_id, Neighbourhood.neigh_name).first()

(1, 'Bijlmer-Centrum')

In [34]:
session.query(Neighbourhood.neigh_id, Neighbourhood.neigh_name).limit(7).all()

[(1, 'Bijlmer-Centrum'),
 (2, 'Bos en Lommer'),
 (3, 'Buitenveldert - Zuidas'),
 (4, 'Centrum-Oost'),
 (5, 'Centrum-West'),
 (6, 'De Aker - Nieuw Sloten'),
 (7, 'De Baarsjes - Oud-West')]

In [35]:
for item in session.query(Room_type):
    print(item.room_type_name, item.room_type_id)

Entire home/apt 1
Hotel room 2
Private room 3
Shared room 4


In [36]:
session.query(Listing).limit(1).one()

Listing(listing_name='Studio with private bathroom in the centre 1', listing_url='https://www.airbnb.com/rooms/20168', amenities='["Wifi", "Hot water", "Hangers", "Host greets you", "Long term stays allowed", "Carbon monoxide alarm", "Fire extinguisher", "Dedicated workspace", "Paid parking off premises", "Essentials", "Bed linens", "Hair dryer", "TV", "Heating", "Refrigerator", "Free street parking", "Smoke alarm"]', bedrooms='1', price='236.00')

In [37]:
session.query(Listing.listing_id, Listing.neighbourhood_id, Listing.host_id).limit(3).all()

[(20168, 4, 59484), (27886, 5, 97647), (28871, 4, 124245)]

In [38]:
session.query(Host.host_id, Host).limit(3).all()

[(47517, Host(host_id='Geert Alexander')),
 (58458, Host(host_id='Dre')),
 (59484, Host(host_id='Alexander'))]

Другие методы: *scalar()*

## 6.4.1. Ограничение числа строк запроса

В предыдущих примерах мы использовали метод *first()*, чтобы вернуть только одну строку. Хотя наш *query()* дал нам одну запрошенную строку, фактический запрос прошел и получил доступ ко всем результатам, а не только к отдельной записи. Если мы хотим ограничить запрос, мы можем использовать нотацию среза массива вместо оператора *limit*.

In [39]:
session.query(Listing.listing_id, 
              Listing.room_type_id,
              Listing.property_type_id,
              Listing.neighbourhood_id,
              Listing.host_id
             )[:4]

[(20168, 3, 31, 4, 59484),
 (27886, 3, 26, 5, 97647),
 (28871, 3, 16, 4, 124245),
 (29051, 3, 16, 4, 124245)]

## 6.4.2. Упорядочивание

Упорядочивание по возрастанию

In [40]:
session.query(Listing.listing_id, 
              Listing.listing_name,
              Listing.neighbourhood_id
             ).order_by(Listing.listing_name)[:4]

[(29480289, '"I cannot stop bragging how good it is" Apartment', 5),
 (5801370, "'Bike' Room Canal District A'dam", 5),
 (37585995, "'Off the grid' stylish typical ADAM 'North' house", 12),
 (34227246, '(#2) Superb Guest room & Balcony (Long-term only)', 7)]

Упорядочивание по убыванию

In [41]:
for item in session.query(Listing.listing_id, 
                  Listing.listing_name,
                  Listing.neighbourhood_id
                 ).order_by(Listing.listing_name.desc())[:4]:
    print('{} - {}'.format(item.listing_name, item.neighbourhood_id))

🏡 Modern and cozy townhouse with green garden - 2
⭐ LUXURIOUS & MODERN APARTMENT AMSTERDAM - 13
☆stylish and spacious Apt in great area w balcony☆ - 20
☆Lovely & spacious  2 persons apt in De Pijp☆ - 8


## 6.4.3. Встроенные функции

SQLAlchemy также может использовать встроенные функции SQL, поддерживаемые базой данных. Две наиболее часто используемые функции - это *sum()* и *count()*. Чтобы использовать эти функции нужно импортировать модуль *sqlalchemy.func*.

In [42]:
from sqlalchemy import func
rec_sum=session.query(func.sum(Listing.price)).scalar()
print(rec_sum)

557116.00


Метод *scalar()* возвращает значение крайнего левого столбца первой строки. В предыдущем примере результатом является одно число. Если использовать метод *first()* для аналогичного запроса, то результатом станет кортеж.

In [43]:
rec_count=session.query(func.count(Listing.listing_id)).first()
print(rec_count)

(3532,)


Результат функции содержится в столбце с автоматически сформированным именем, таким как *count_1*. Чтобы присовить столбцу результата другое имя и использовать его в дальнейшем, испольльзуем функцию *label().*

In [44]:
rec_count=session.query(func.sum(Listing.price).label('overall_price')).first()
print(rec_count)
print(rec_count.keys())
print(rec_count.overall_price)

(Decimal('557116.00'),)
['overall_price']
557116.00


## 6.4.4. Фильтрация

Фильтрация осуществляется посредством "пристегивания" к запросу выражения фильтрации с помощью метода *filter()*. Обычно такое выражение содержит столбец, оператор и значение или другой столбец. Можно прикреплять к запросу несколько фильтров или перечислять условия фильтрации через запятую в одном фильтре. Для фильтрации используют методы *filter()* и *filter_by*.

## Метод *filter()*

In [45]:
from pprint import pprint
record=session.query(Listing.listing_id, Listing).filter(Listing.bedrooms==6).all()
pprint(record)

[(2117442,
  Listing(listing_name='Classic sailing ship in Amsterdam', listing_url='https://www.airbnb.com/rooms/2117442', amenities='["Oven", "Iron", "Hot water", "Private entrance", "Host greets you", "Long term stays allowed", "Fire extinguisher", "Luggage dropoff allowed", "Dedicated workspace", "Dryer", "Essentials", "Kitchen", "First aid kit", "Hair dryer", "Heating", "Washer", "Smoke alarm", "Stove"]', bedrooms='6', price='550.00')),
 (8142900,
  Listing(listing_name='Historical Canal House near ANNE FRANK House', listing_url='https://www.airbnb.com/rooms/8142900', amenities='["Children\u2019s books and toys", "Microwave", "Carbon monoxide alarm", "First aid kit", "Refrigerator", "Dishes and silverware", "Smoke alarm", "High chair", "Oven", "Children\u2019s dinnerware", "Private entrance", "Babysitter recommendations", "Cable TV", "TV", "Stove", "Wifi", "Pocket wifi", "Hot water", "Coffee maker", "Hangers", "Shampoo", "Bathtub", "Luggage dropoff allowed", "Kitchen", "Cooking bas

In [46]:
record=session.query(Listing.listing_id, Listing) \
    .filter(Listing.bedrooms==6) \
    .filter(Listing.price>600) \
    .all()
pprint(record)

[(15542104,
  Listing(listing_name='Beautiful Villa Oud-Zuid/Centre, 6 bedrooms 1500m3', listing_url='https://www.airbnb.com/rooms/15542104', amenities='["Children\u2019s books and toys", "Microwave", "Carbon monoxide alarm", "First aid kit", "Refrigerator", "Dishes and silverware", "Smoke alarm", "Game console", "Oven", "Patio or balcony", "Children\u2019s dinnerware", "Babysitter recommendations", "Cable TV", "TV", "Stove", "Wifi", "Pocket wifi", "Hot water", "Coffee maker", "Hangers", "Fire extinguisher", "Shampoo", "Bathtub", "Luggage dropoff allowed", "Kitchen", "Cooking basics", "Indoor fireplace", "Garden or backyard", "Iron", "Room-darkening shades", "Dishwasher", "Dedicated workspace", "Paid parking off premises", "Dryer", "Essentials", "Heating", "Washer"]', bedrooms='6', price='625.00'))]


## Метод *filter_by()*

Метод *filter_by()* использует несколько иной синтаксис. Имя класса не используется, вместо этого используется атрибут главного класса запроса, либо класса, который последним был присоединен к запросу. Также вместо оператора сравнения этот метод использует оператор присваивания.

```
record=session.query(Listing.listing_id, Listing) \
    .filter_by(bedrooms=4, price=679).all()
print(record)
```

## Условные методы

In [47]:
CM=pd.read_csv('./Data/ClauseMethods.csv', sep=';')
CM

,Метод,Описание
0,"between(cleft,cright)",Поиск значения между cleft и cright
1,concat(column_two),Соединяет столбкц со столбцом 2
2,distinct(),Вывод без повторов
3,in_([list]),Если значение находится в списке
4,is_(None),Проверка на пустое значение
5,contains(string),Если значение содержит подстроку (регистр)
6,endswith(string),Оканчивается на подстроку
7,like(string),Поиск по шаблону (зависит от регистра)
8,startswith(string),Начинается с
9,ilike(string),Поиск по шаблону (не зависит от регистра)


In [48]:
session.query(Host.host_id, Host.host_name) \
    .filter(Host.host_name.like('%Alexander%')).all()

[(47517, 'Geert Alexander'),
 (59484, 'Alexander'),
 (17873793, 'Alexander'),
 (20766093, 'Alexander'),
 (23593514, 'Alexander'),
 (25084932, 'Alexander'),
 (26045647, 'Alexander'),
 (72822537, 'Alexander'),
 (207287996, 'Alexander')]

## Логические связки

### *and_(), or_(), not_()*

In [49]:
from sqlalchemy import and_, or_, not_

session.query(Listing.listing_id, Listing).filter(
    and_(
        Listing.bedrooms>6,
        Listing.price.between(600, 700)
    )
).all()

[(2026455,
  Listing(listing_name='Sailing ship Isis.', listing_url='https://www.airbnb.com/rooms/2026455', amenities='["Oven", "Hot water", "Coffee maker", "Fire extinguisher", "Essentials", "Dryer", "First aid kit", "Kitchen", "Dishes and silverware", "Heating", "Cooking basics", "Washer", "Refrigerator", "Smoke alarm", "Stove"]', bedrooms='9', price='636.00'))]

## 6.4.5. Обновление. Метод *update()*

Рассмотрим два способа обновления данных. Найдем в базе данных объект размещения с ценой 679.

In [50]:
session.query(Listing.listing_id, Listing).filter(Listing.price==679).all()

[(1237639,
  Listing(listing_name='Apartment 1653', listing_url='https://www.airbnb.com/rooms/1237639', amenities='["Wifi", "Iron", "Microwave", "Dishwasher", "Essentials", "Kitchen", "Hair dryer", "TV", "Heating", "Cooking basics", "Refrigerator", "Stove"]', bedrooms='4', price='679.00'))]

Увеличим цену на 1.

In [51]:
q = session.query(Listing)
w = q.filter(Listing.listing_id == 1237639).one()
print(w)
w.price = w.price + 1
session.commit()

print(w.price)

Listing(listing_name='Apartment 1653', listing_url='https://www.airbnb.com/rooms/1237639', amenities='["Wifi", "Iron", "Microwave", "Dishwasher", "Essentials", "Kitchen", "Hair dryer", "TV", "Heating", "Cooking basics", "Refrigerator", "Stove"]', bedrooms='4', price='679.00')
680.00


И еще на 1.

In [52]:
q = session.query(Listing)
q = q.filter(Listing.listing_id == 1237639)
q.update({Listing.price: Listing.price + 1})

print(q.first())

Listing(listing_name='Apartment 1653', listing_url='https://www.airbnb.com/rooms/1237639', amenities='["Wifi", "Iron", "Microwave", "Dishwasher", "Essentials", "Kitchen", "Hair dryer", "TV", "Heating", "Cooking basics", "Refrigerator", "Stove"]', bedrooms='4', price='681.00')


## 6.4.6.Удаление. Метод *delete()*

Рассмотрим способы удаления данных. Найдём все объекты, в названии которых имеется фраза "bright & quiet".

In [53]:
filter_condition=Listing.listing_name.like('%bright & quiet%')

session.query(Listing.listing_id, Listing.listing_name) \
    .order_by(Listing.listing_id) \
    .filter(filter_condition).all()

[(9462244, 'Modern, bright & quiet apartment'),
 (33677630, 'Bright & quiet home with garden, close to centre')]

Удалим все такие записи

In [54]:
# 1
q = session.query(Listing)
q = q.filter(filter_condition)
records_to_del = q.all()
for record in records_to_del:
    session.delete(record)
session.commit()

Проверим, остались ли записи, соответствующие условию

In [55]:
q=session.query(Listing)
q=q.filter(Listing.listing_name.like(filter_condition))
q.all()

[]

## 6.4.7. Соединение таблиц. Методы *join* и *outerjoin*

Найдем все объекты размещения, заказанные пользователем *Vera*. Как мог бы выглядеть поиск этих объектов без *join?* Например, так:

In [56]:
q=session.query(Listing.listing_id, Listing.listing_name)
q=q.filter(Listing.listing_id==Line_item.listing_id)
q=q.filter(Line_item.order_id==Order.order_id)
q=q.filter(Order.user_id==User.user_id)
q=q.filter(User.username=='Vera')

q.all()

[(2550571, 'Cosy, Quiet & Stylish close to Jordaan and Centre!'),
 (18225467, 'Comfortable bright family house, close to ferry'),
 (3908795, 'Gorgeous House near Vondelpark area')]

Или так:

In [57]:
q4=session.query(User.user_id).filter(User.username=='Vera')
q3=session.query(Order.order_id).filter(Order.user_id==q4)
q2=session.query(Line_item.listing_id).filter(Line_item.order_id.in_(q3))
q1=session.query(Listing.listing_id, Listing.listing_name)
q1=q1.filter(Listing.listing_id.in_(q2))

q1.all()

[(2550571, 'Cosy, Quiet & Stylish close to Jordaan and Centre!'),
 (3908795, 'Gorgeous House near Vondelpark area'),
 (18225467, 'Comfortable bright family house, close to ferry')]

Выполним этот запрос с использованием соединения *join*

In [58]:
q=session.query(Listing.listing_id, Listing.listing_name)
q=q.join(Line_item).join(Order).join(User)
q=q.filter(User.username=='Vera')

q.all()

[(2550571, 'Cosy, Quiet & Stylish close to Jordaan and Centre!'),
 (18225467, 'Comfortable bright family house, close to ferry'),
 (3908795, 'Gorgeous House near Vondelpark area')]

## 6.4.8. Группировка. Метод *group_by*

Найдём, сколько объектов заказали клиенты.

In [59]:
q=session.query(User.username, func.count(Listing.listing_id).label('cnt'))
q=q.join(Order).join(Line_item).join(Listing)
q=q.group_by(User.user_id)
print(str(q))
q.all()

SELECT users.username AS users_username, count(listings.listing_id) AS cnt 
FROM users JOIN orders ON users.user_id = orders.user_id JOIN line_items ON orders.order_id = line_items.order_id JOIN listings ON listings.listing_id = line_items.listing_id GROUP BY users.user_id


[('Nicolas', 3), ('Lida', 1), ('Vera', 3), ('Ivan', 2)]

В нашей  базе данных есть пользователь Svetlana с user_id=5, который разместил четыре заказа:

In [60]:
sv_orders=session.query(Order.order_id).filter(Order.user_id==5)
sv_orders.all()

[(9), (10), (11), (12)]

при этом не сделал ни одной детализации, т.е. не указал конкрентых объектов:

In [61]:
sv_items=session.query(func.count(Line_item.listing_id))
sv_items=sv_items.filter(Line_item.order_id.in_(sv_orders))
sv_items.scalar()

0

С помощью *outerjoin* распечатаем число объектов для каждого клиента, включая клиентов, которые не детализировали ни однго заказа.

In [62]:
q=session.query(User.username, func.count(Listing.listing_id).label('cnt'))
q=q.join(Order).outerjoin(Line_item).outerjoin(Listing)
q=q.group_by(User.user_id)
print(str(q))
q.all()

SELECT users.username AS users_username, count(listings.listing_id) AS cnt 
FROM users JOIN orders ON users.user_id = orders.user_id LEFT OUTER JOIN line_items ON orders.order_id = line_items.order_id LEFT OUTER JOIN listings ON listings.listing_id = line_items.listing_id GROUP BY users.user_id


[('Nicolas', 3), ('Lida', 1), ('Vera', 3), ('Ivan', 2), ('Svetlana', 0)]

## 6.4.9. Прямые запросы

In [63]:
from sqlalchemy import text

In [64]:
q=session.query(User).filter(text("username=='Lida'"))
print(q.all())

[User(username='Lida', email_address='lidaok@gmail.com', phone='+7-929-616-88-77', password='yyT$%333')]


<a id=T_3_5></a>
[<= ](#T_3_4)||[ К оглавлению ](#Ref)||[ =>](#T_3_6)

# 6.5. Отображение

Чтобы отобразить базу данных, вместо класса *declarative_base* будем использовать базовый класс автосопоставления *automap_base*. Начнем с создания базового объекта *Base.*

In [2]:
from sqlalchemy.ext.automap import automap_base
Base = automap_base()

Теперь нам необходим механизм соединения с базой данных, которую мы хотим отобразить

In [3]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///Data/Students_2021.sqlite')

С базовым объектом и настройкой движка у нас есть все необходимое для отображения базы данных. Метода *prepare* объекта *Base* просканирует все, что доступно в движке, который мы только что создали, и сделает отображение.

In [4]:
Base.prepare(engine, reflect=True)

Эта строка кода - все, что нужно, чтобы отобразить всю базу данных! Теперь объекты ORM для каждой таблицы доступны в свойстве *classes* объекта *Base*.

In [5]:
Base.classes.keys()

['city',
 'exam_marks',
 'student',
 'university',
 'subject',
 'lecturer',
 'subj_lect']

Если к отображенной базе данных нужно добавить таблицу. Добавление таблицы вручную. Пример.
```
from sqlalchemy import (Table, Column, 
                        Integer, Numeric, String, Boolean,
                        ForeignKey, ForeignKeyConstraint, CheckConstraint)

from datetime import datetime
from sqlalchemy import DateTime

# Инструкцию Base = declarative_base() не используем.
# Вместо этого используем созданный выше Base = automap_base(). 
# Новый Base не создаем, так как нам надо добавить таблицу 
# не в новую, а в уже отображенную БД

class Subj_lect(Base):
    __tablename__ = 'subj_lect'

    subj_lect_id = Column(Integer(), primary_key = True)
    lecturer_id = Column(Integer())
    subj_id = Column(Integer())
    
    __table_args__ = (
        ForeignKeyConstraint(['lecturer_id'],['lecturer.lecturer_id']),
        ForeignKeyConstraint(['subj_id'], ['subject.subj_id']),
    )

Base.metadata.create_all(engine)
```

Создадим объекты для ссылок на таблицы

In [6]:
city,exam_marks,student,university,subject,lecturer,subj_lect = Base.classes.values()
#Album,Artist,Customer,Employee,Genre,Invoice,InvoiceLine,Track,MediaType,Playlist = Base.classes.values()

Выполним запрос. Распечатаем информацию о предметах

In [7]:
from sqlalchemy.orm import Session
session = Session(engine)

In [8]:
for record in session.query(subject).all():
    print(record.subj_id, record.subj_name, record.hour)

10 Информатика 56
11 Программирование 34
12 Анализ данных 42
13 ОБЖ 34
22 Физика 34
43 Математика 56
56 История 34
73 Физкультура 34
94 Английский 56


Распечатаем имена, фамилии преподавателей и названия предметов, которые они ведут. Для каждого использования метода *join* уточним атрибуты соединения.

In [9]:
q=session.query(lecturer.name, lecturer.surname, subject.subj_name)
q=q.join(subj_lect, lecturer.lecturer_id==subj_lect.lecturer_id)
q=q.join(subject, subj_lect.subj_id==subject.subj_id)
q.all()

[('Лукия', 'Сафонова', 'Информатика'),
 ('Лукия', 'Сафонова', 'Программирование'),
 ('Юлия', 'Зайцева', 'Анализ данных'),
 ('Акулина', 'Мамонтова', 'Физика'),
 ('Павел', 'Лагутин', 'История'),
 ('Иванна', 'Максимова', 'История'),
 ('Михаил', 'Волков', 'Физкультура'),
 ('Оксана', 'Богданова', 'Анализ данных'),
 ('Оксана', 'Богданова', 'Математика')]

Автосопоставление *Automap* может автоматически отображать и устанавливать отношения «много к одному», «один ко многим» и «много ко многим» посредством создания свойства `<related_object>_collection`.

In [ ]:
lecturer.__table__

In [ ]:
university.__table__

In [10]:
u = session.query(university).filter(university.univ_id==46).one()
for l in u.lecturer_collection:
    print(u.univ_name, l.name, l.surname)

Политех Лукия Сафонова
Политех Руслан Стрелков
Политех Юлиан Зимин


Дополнительно об отображении данных в <a href="https://docs.sqlalchemy.org/en/14/orm/extensions/automap.html#specifying-classes-explcitly">документации</a>

<a id=T_3_6></a>
[<= ](#T_3_5)||[ К оглавлению ](#Ref)||[ =>](#T_3_7)

# 6.6. Сеанс

Когда мы используем запрос для получения объекта, мы возвращаем объект, связанный с сеансом. Этот объект может перемещаться через несколько состояний по отношению к сеансу.Существует четыре возможных состояния экземпляров объекта данных:
- Transient (временное): экземпляр не находится в сеансе и отсутствует в базе данных.
- Pending (в ожидании): экземпляр был добавлен в сеанс с помощью *add()*, но не был сброшен или зафиксирован.
- Persistent (постоянное): объект в сеансе имеет соответствующую запись в базе данных.
- Detached (отсоединенное): экземпляр больше не подключен к сеансу, но имеет запись в базе данных.

Мы можем наблюдать, как экземпляр проходит свои состояния. Выполним отображение *Listings.db*

In [12]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from datetime import datetime
engine = create_engine('sqlite:///Listings.db')
Base = automap_base()
Base.prepare(engine, reflect=True)
session = Session(engine)

In [13]:
Base.classes.keys()

['hosts',
 'line_items',
 'orders',
 'users',
 'listings',
 'neighbourhoods',
 'property_types',
 'room_types']

In [14]:
listings = Base.classes.listings
line_items = Base.classes.line_items
orders = Base.classes.orders
users = Base.classes.users

Создадим экземпляр детализации заказа пользователя Svetlana. Вспомним сначала, какие заказы она разместила.

In [15]:
sv_orders = session.query(orders).filter(orders.user_id==5).all()
for item in sv_orders:
    print(item.order_id, item.created_on)

9 2020-05-09 00:00:00
10 2020-05-10 00:00:00
11 2021-05-09 00:00:00
12 2021-05-09 00:00:00


Создадим детализацию заказа № 9. Пусть это будет объект 29051 Amsterdam Center Entire Apartment в период с 1 по 11 мая 2021 года.

In [16]:
a_line_item = line_items(order_id = int(9),
                         listing_id = int(29051),
                         item_start_date = datetime.strptime('2021-05-01','%Y-%m-%d'),
                         item_end_date = datetime(2021,5,11)
                        )

Чтобы увидеть состояние экземпляра, применим метод SQLAlchemy *inspect()*.

In [11]:
from sqlalchemy import inspect
insp = inspect(a_line_item)

Теперь мы можем обратиться к свойствам объекта *insp: transient, pending, persistent, detached*.

In [25]:
print(insp.transient, insp.pending,)

False False


Пройдем в цикле по всем свойствам.

In [15]:
for state in ['transient', 'pending', 'persistent', 'detached']:
    print('{:>10}: {}'.format(state, getattr(insp, state)))

 transient: True
   pending: False
persistent: False
  detached: False


Как видим в выходных данных, текущее состояние нашего экземпляра детализации заказов является временным, то есть в состоянии, в котором находятся вновь созданные объекты до того, как они будут сброшены или зафиксированы в базе данных. Если мы добавим *a_line_item* в текущий сеанс и повторно запросим состояние, мы получим следующий результат:

In [18]:
session.add(a_line_item)

In [19]:
for state in ['transient', 'pending', 'persistent', 'detached']:
    print('{:>10}: {}'.format(state, getattr(insp, state)))

 transient: False
   pending: True
persistent: False
  detached: False


Состояние изменилось. Подтвердим изменения.

In [20]:
session.commit()

In [21]:
for state in ['transient', 'pending', 'persistent', 'detached']:
    print('{:>10}: {}'.format(state, getattr(insp, state)))

 transient: False
   pending: False
persistent: True
  detached: False


Для демонстрации отсоединенного состояния выполним метод *expunge,* который используется для удаления объекта из сессии.

In [23]:
session.expunge(a_line_item)

In [24]:
for state in ['transient', 'pending', 'persistent', 'detached']:
    print('{:>10}: {}'.format(state, getattr(insp, state)))

 transient: False
   pending: False
persistent: False
  detached: True


Посмотрим теперь, как использовать  инспектор для отображения истории изменений. Вернём объект в сессию и изменим номер объекта. Пусть теперь это объект 41125 Amsterdam Center Entire Apartment.

In [26]:
session.add(a_line_item)
a_line_item.listing_id = 41125

Применим свойство инспектора *modified*

In [27]:
insp.modified

True

Мы получили значение истины. Используем теперь коллекцию *attr*, чтобы узнать, что было изменено.

In [29]:
for attr, attr_state in insp.attrs.items():
    if attr_state.history.has_changes(): # Проверка состояния атрибута
        print('{}: {}'.format(attr, attr_state.value))
        print('History: {}\n'.format(attr_state.history)) # История измененного объекта

listing_id: 41125
History: History(added=[41125], unchanged=(), deleted=())



В результате мы получили данные об изменении идентификатора объекта. На этом примере мы изучили, как объекты взаимодействуют с сессией.